## Import

In [1]:
import random
import pandas as pd
import numpy as np
import os
from PIL import Image

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

from torchvision import transforms
import torchvision.models as models

from tqdm.auto import tqdm

import warnings
warnings.filterwarnings(action='ignore')

In [2]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

## Hyperparameter Setting

In [3]:
CFG = {
    'IMG_SIZE':320,
    'EPOCHS':10,
    'LEARNING_RATE':1e-3,
    'BATCH_SIZE':64,
    'SEED':41
}

## Fixed RandomSeed

In [4]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True

seed_everything(CFG['SEED']) # Seed 고정

## Data Load

In [5]:
train_df = pd.read_csv('./train0.csv')
test_df = pd.read_csv('./test.csv')

## Data Preprocessing

In [6]:
# Train / Validation Split (70% : 30%)
train_len = int(len(train_df) * 0.7)
val_df = train_df.iloc[train_len:]
train_df = train_df.iloc[:train_len]

In [7]:
train_labels = train_df.iloc[:,2:].values.reshape(-1, 4, 4)
val_labels = val_df.iloc[:,2:].values.reshape(-1, 4, 4)

In [8]:
(train_labels.shape, val_labels.shape)

((146998, 4, 4), (63000, 4, 4))

## CustomDataset

In [9]:
class CustomDataset(Dataset):
    def __init__(self, img_path_list, label_list, transform=None):
        self.img_path_list = img_path_list
        self.label_list = label_list
        self.transform = transform

    def __getitem__(self, index):
        img_path = self.img_path_list[index]

        # PIL 이미지로 불러오기
        image = Image.open(img_path).convert("RGB")
        if self.transform is not None:
            image = self.transform(image)

        if self.label_list is not None:
            label = torch.tensor(self.label_list[index], dtype=torch.long) - 1
            return image, label
        else:
            return image

    def __len__(self):
        return len(self.img_path_list)

In [10]:
train_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Resize((CFG['IMG_SIZE'], CFG['IMG_SIZE'])),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

test_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Resize((CFG['IMG_SIZE'], CFG['IMG_SIZE'])),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

In [11]:
train_dataset = CustomDataset(train_df['img_path'].values, train_labels, train_transform)
train_loader = DataLoader(train_dataset, batch_size = CFG['BATCH_SIZE'], shuffle=True, num_workers=0)

val_dataset = CustomDataset(val_df['img_path'].values, val_labels, test_transform)
val_loader = DataLoader(val_dataset, batch_size = CFG['BATCH_SIZE'], shuffle=False, num_workers=0)

test_dataset = CustomDataset(test_df['img_path'].values, None, test_transform)
test_loader = DataLoader(test_dataset, batch_size = CFG['BATCH_SIZE'], shuffle=False, num_workers=0)

In [13]:
import torch
import torch.nn as nn
import torch.nn.functional as F


class UNet3(nn.Module):
    def __init__(self,DSV=True):
        super(UNet3,self).__init__()

        self.DSV = DSV
        def cbr(in_channels, out_channels, kernel_size = 3, stride = 1):
            layers = nn.Sequential(
                nn.Conv2d(in_channels,out_channels,kernel_size,stride,1),
                #nn.BatchNorm2d(out_channels),
                nn.ReLU(),

                nn.Conv2d(out_channels,out_channels,kernel_size,stride,1),
                #nn.BatchNorm2d(out_channels),
                nn.ReLU()
            )
            return layers

        self.h = nn.Sequential(
            nn.Conv2d(320,320,3,1,1),
            nn.BatchNorm2d(320),
            nn.ReLU(),

            nn.Conv2d(320,320,3,1,1),
            nn.BatchNorm2d(320),
            nn.ReLU()
        )


        self.enc0 = cbr(1,64)
        self.enc1 = cbr(64,128)
        self.enc2 = cbr(128,256)
        self.enc3 = cbr(256,512)
        self.enc4 = cbr(512,1024)

        self.pool = nn.MaxPool2d(2,2)

        self.enc0_dec0 = nn.Sequential(
            nn.Conv2d(64,64,3,1,1)
        )
        self.enc0_dec1 = nn.Sequential(
            nn.MaxPool2d(2,2),
            nn.Conv2d(64,64,3,1,1)
        )
        self.enc0_dec2 = nn.Sequential(
            nn.MaxPool2d(4,4),
            nn.Conv2d(64,64,3,1,1)
        )
        self.enc0_dec3 = nn.Sequential(
            nn.MaxPool2d(8,8),
            nn.Conv2d(64,64,3,1,1)
        )

        self.enc1_dec1 = nn.Sequential(
            nn.Conv2d(128,64,3,1,1)
        )
        self.enc1_dec2 = nn.Sequential(
            nn.MaxPool2d(2,2),
            nn.Conv2d(128,64,3,1,1)
        )
        self.enc1_dec3 = nn.Sequential(
            nn.MaxPool2d(4,4),
            nn.Conv2d(128,64,3,1,1)
        )

        self.enc2_dec2 = nn.Sequential(
            nn.Conv2d(256,64,3,1,1)
        )
        self.enc2_dec3 = nn.Sequential(
            nn.MaxPool2d(2,2),
            nn.Conv2d(256,64,3,1,1)
        )

        self.enc3_dec3 = nn.Sequential(
            nn.Conv2d(512,64,3,1,1)
        )


        self.dec_up2 = nn.Sequential(
            nn.Upsample(scale_factor=2, mode='bilinear'),
            nn.Conv2d(320,64,3,1,1)
        )

        self.dec_up4 = nn.Sequential(
            nn.Upsample(scale_factor=4, mode='bilinear'),
            nn.Conv2d(320,64,3,1,1)
        )

        self.dec_up8 = nn.Sequential(
            nn.Upsample(scale_factor=8, mode='bilinear'),
            nn.Conv2d(320,64,3,1,1)
        )

        self.enc4_up2 = nn.Sequential(
            nn.Upsample(scale_factor=2, mode='bilinear'),
            nn.Conv2d(1024,64,3,1,1)
        )

        self.enc4_up4 = nn.Sequential(
            nn.Upsample(scale_factor=4, mode='bilinear'),
            nn.Conv2d(1024,64,3,1,1)
        )

        self.enc4_up8 = nn.Sequential(
            nn.Upsample(scale_factor=8, mode='bilinear'),
            nn.Conv2d(1024,64,3,1,1)
        )

        self.enc4_up16 = nn.Sequential(
            nn.Upsample(scale_factor=16, mode='bilinear'),
            nn.Conv2d(1024,64,3,1,1)
        )

        self.result1 = nn.Sequential(
            nn.Conv2d(320,1,3,1,1),
            nn.Sigmoid()
        )
        self.result2 = nn.Sequential(
            nn.Conv2d(1024,1,3,1,1),
            nn.Sigmoid()
        )




    def forward(self,x):

        enc0 = self.enc0(x)
        pool1 = self.pool(enc0)

        enc1 = self.enc1(pool1)
        pool2 = self.pool(enc1)

        enc2 = self.enc2(pool2)
        pool3 = self.pool(enc2)

        enc3 = self.enc3(pool3)
        pool4 = self.pool(enc3)

        enc4 = self.enc4(pool4)

        dec3 = self.h(torch.cat([self.enc0_dec3(enc0),self.enc1_dec3(enc1),self.enc2_dec3(enc2),self.enc3_dec3(enc3),self.enc4_up2(enc4)],1))
        dec2 = self.h(torch.cat([self.enc0_dec2(enc0),self.enc1_dec2(enc1),self.enc2_dec2(enc2),self.dec_up2(dec3),self.enc4_up4(enc4)],1))
        dec1 = self.h(torch.cat([self.enc0_dec1(enc0),self.enc1_dec1(enc1),self.dec_up2(dec2),self.dec_up4(dec3),self.enc4_up8(enc4)],1))
        dec0 = self.h(torch.cat([self.enc0_dec0(enc0),self.dec_up2(dec1),self.dec_up4(dec2),self.dec_up8(dec3),self.enc4_up16(enc4)],1))

        out = []
        if self.DSV:
            out.append(self.result1(dec0))
            out.append(self.result1(dec1))
            out.append(self.result1(dec2))
            out.append(self.result1(dec3))
            out.append(self.result2(enc4))
        else:
            out.append(self.result1(dec0))

        return out



## Train

In [14]:
def train(model, optimizer, train_loader, val_loader, device):
    model.to(device)
    criterion = nn.CrossEntropyLoss().to(device)

    best_val_acc = 0
    best_model = None
    for epoch in range(1, CFG['EPOCHS']+1):
        model.train()
        train_loss = []
        for imgs, labels in tqdm(iter(train_loader)):
            imgs = imgs.float().to(device)
            labels = labels.to(device)

            optimizer.zero_grad()

            output = model(imgs)
            loss = criterion(output, labels)

            loss.backward()
            optimizer.step()

            train_loss.append(loss.item())

        _val_loss, _val_acc = validation(model, criterion, val_loader, device)
        _train_loss = np.mean(train_loss)
        print(f'Epoch [{epoch}], Train Loss : [{_train_loss:.5f}] Val Loss : [{_val_loss:.5f}] Val ACC : [{_val_acc:.5f}]')

        if best_val_acc < _val_acc:
            best_val_acc = _val_acc
            best_model = model

    return best_model

In [15]:
def validation(model, criterion, val_loader, device):
    model.eval()
    val_loss = []
    val_acc = []

    with torch.no_grad():
        for imgs, labels in tqdm(iter(val_loader)):
            imgs = imgs.float().to(device)
            labels = labels.to(device)

            output = model(imgs)

            loss = criterion(output, labels)

            val_loss.append(loss.item())

            # 정확도 계산을 위한 예측 레이블 추출
            predicted_labels = torch.argmax(output, dim=1)

            # 샘플 별 정확도 계산
            for predicted_label, label in zip(predicted_labels, labels):
                val_acc.append(((predicted_label == label).sum() / 16).item())

        _val_loss = np.mean(val_loss)
        _val_acc = np.mean(val_acc)

    return _val_loss, _val_acc

## Run!!

In [16]:
model = UNet3()
optimizer = torch.optim.Adam(params = model.parameters(), lr = CFG["LEARNING_RATE"])
infer_model = train(model, optimizer, train_loader, val_loader, device)

  0%|          | 0/2297 [00:00<?, ?it/s]

RuntimeError: Given groups=1, weight of size [64, 1, 3, 3], expected input[64, 3, 320, 320] to have 1 channels, but got 3 channels instead

## Inference

In [ ]:
def inference(model, test_loader, device):
    model.eval()
    preds = []
    with torch.no_grad():
        for imgs in tqdm(iter(test_loader)):
            imgs = imgs.float().to(device)

            output = model(imgs)

            # 정확도 계산을 위한 예측 레이블 추출
            predicted_labels = torch.argmax(output, dim=1).view(-1, 16)
            predicted_labels = predicted_labels.cpu().detach().numpy()

            preds.extend(predicted_labels)

    return preds

In [ ]:
preds = inference(infer_model, test_loader, device)

## Submission

In [ ]:
submit = pd.read_csv('./sample_submission.csv')

In [ ]:
submit.iloc[:, 1:] = preds
submit.iloc[:, 1:] += 1

In [ ]:
submit.to_csv('./baseline_submit.csv', index=False)